## Make connection

In [1]:
import paramiko, os, re
from getpass import getpass

# Create the passwd prompt
thePass = getpass(prompt="Please enter your SSH Password: ")


# Host Information
host = "192.168.6.71"
port = 2222
username = "paul.gleason"
password = thePass

try:
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy)
    ssh.connect(host, port, username, password)

except paramiko.AuthenticationException:
    print("Authentication Failed")

## Check Directory

In [2]:
stdin, stdout, stderr = ssh.exec_command('pwd')
print(stdout.readlines())

['/home/paul.gleason\n']


## Place kraken file and verify it's in the right location with ls

In [3]:
sftp = ssh.open_sftp()

ssh.exec_command('cd /home/paul.gleason/')

sftp.put('kraken', 'kraken') 

stdin, stdout, stderr = ssh.exec_command('ls')
print(stdout.readlines())


['kraken\n']


## Run kraken file and write output to kraken.txt

In [2]:
if os.path.exists('pid.txt'):
    usrinput = input("Would you like to delete this file? (Y/N) ")
    if usrinput == "Y" or usrinput == "y":
        os.remove("pid.txt")

thePass = getpass(prompt="Please enter your user Password: ")

stdin, stdout, stderr = ssh.exec_command("sudo -S /home/ubuntu/kraken --folder /usr/bin --folder /usr/sbin/   --folder /usr/local/bin  --folder /sbin  --folder /usr/local/sbin  --folder /bin")

stdin.write(thePass + "\n")
stdin.flush()

# for line in stdout.readlines():
#     with open('pg.pid', 'a') as f:
#         f.write(line)

for line in stdout.readlines():
    if re.findall(r'.*pid.*', line):
        with open('pid.txt', 'a') as f:
            f.write(line.split(" ")[-2].split("=")[-1] + ' ')


## Put lsos_run.sh in home dir

In [ ]:
sftp = ssh.open_sftp()

ssh.exec_command('cd /home/paul.gleason/')

sftp.put('lsof_run.sh', 'lsof_run.sh') 

stdin, stdout, stderr = ssh.exec_command('chmod +x ./lsof_run.sh')

stdin, stdout, stderr = ssh.exec_command('ls -la')
for line in stdout.readlines():
    print(line)

In [3]:
sftp = ssh.open_sftp()

sftp.put('pid.txt', 'pid.txt') 

stdin, stdout, stderr = ssh.exec_command('./lsof_run.sh')
for line in stdout.readlines():
    print(line)

## lsof -p XXXX to just get columm 9

In [4]:
lines = []

thePass = getpass(prompt="Please enter your user Password: ")

with open('pid.txt', 'r') as f:
    for line in f:
        lines.append(line)

for item in lines:
    split_lines = item.split(" ")

for t in split_lines:
    # print(t)

    command = "sudo -S lsof -p " + t + " | awk '{print $9}'"

    # print(command)

    stdin, stdout, stderr = ssh.exec_command(command)
    stdin.write(thePass + "\n")
    stdin.flush()

    for s in stdout.readlines():
        
        print(s)

NAME

/

/

/snap/snapd/17336/usr/lib/snapd/snapd

/usr/lib/x86_64-linux-gnu/libresolv-2.31.so

/usr/lib/x86_64-linux-gnu/libnss_dns-2.31.so

/usr/lib/x86_64-linux-gnu/libnss_files-2.31.so

/usr/lib/x86_64-linux-gnu/libc-2.31.so

/usr/lib/x86_64-linux-gnu/libpthread-2.31.so

/usr/lib/x86_64-linux-gnu/ld-2.31.so

/dev/null

type=STREAM

type=STREAM

type=STREAM

/run/snapd.socket

[eventpoll]

/var/lib/snapd/state.lock

/run/snapd-snap.socket



pipe

pipe

NAME

/home/p.dalton

/

/usr/local/bin/egrep

/usr/lib/x86_64-linux-gnu/libc-2.31.so

/usr/lib/x86_64-linux-gnu/libpthread-2.31.so

/usr/lib/x86_64-linux-gnu/ld-2.31.so

/dev/null

/home/p.dalton/.d.log

/home/p.dalton/.d.log

/dev/null

/home/p.dalton/d.pid

*:http-alt

[eventpoll]

pipe

pipe



## Task #5

### Download any suspicious files in home dirs

In [13]:
stdin, stdout, stderr = ssh.exec_command("ls -al -h k /home/ | awk '{print $9}'")
for username in stdout.readlines():
    stdin, stdout, stderr = ssh.exec_command("ls -al -h k /home/" + username + " | awk '{print $4}'")
    for item in stdout.readlines():
        if item != "domain^users":
            


# if os.path.exists('home_results.txt'):
#     usrinput = input("Would you like to delete this file? (Y/N) ")
#     if usrinput == "Y" or usrinput == "y":
#         os.remove("home_results.txt")

# # Get the outpt form the command
# stdin, stdout, stderr = ssh.exec_command(command)

# split_lines = []

# for line in stdout.readlines():
#     # print(line.split(' '))
#     split_lines.append(line.split(' '))
    
#     # Searches and returns results using a regular expression search

# files = []
# split_command = command.split(' ')

# # print(split_lines)
# # loop through split_lines
# for new_line in split_lines:
#     # if new_line is longer than 2
#     if len(new_line) > 2:
#         # if there is a symlink
#         if new_line[-2] != '->':
#             for item in new_line:
#                 # print(item)
#                 if '\n' in item:
#                     # print(item[0:-2])
#                     files.append(item[0:-2])
                    
#         else:
#             # print(new_line[-3])
#             files.append(new_line[-3])

# # for r in files:
# #     print(r)


# for file in files:
#     x = re.findall(r'^([a-z]{6,17})$', file)
#     for found in x:
#         output = split_command[-1] + found


# # Get results from stdout
# # lines = stdout.readlines()
# # print(lines)

# # Cover the list to a string
# # output = ''.join(lines)

#         # Header output
#         sepHeader = '' + '### BEGIN ' + command + ' ###\n\n'
#         # Footer
#         sepFooter = '' + '### END ' + command + ' ###\n\n'

#         # Concatenate the header, output, and footer
#         cmd_output = sepHeader + output + sepFooter

#         # Sace the cmd_output to a file
#         # a is append mode
#         with open('home_results.txt', 'a') as f:
#             f.write(output + '\n')

/home/:

total 256K

drwxr-xr-x 63 root              root         4.0K Oct 31 18:51 .

drwxr-xr-x 19 root              root         4.0K Oct 31 19:11 ..

-rw-------  1 root              root          510 Mar  8  2022 .d.log

drwx------  2 Acese1965         Acese1965    4.0K Feb 26  2022 Acese1965

drwx------  2 Andneas           Andneas      4.0K Feb 26  2022 Andneas

drwx------  2 Andoged           Andoged      4.0K Feb 26  2022 Andoged

drwx------  2 Asereardscon              1018 4.0K Feb 26  2022 Asereardscon

drwx------  2 Denestomed        Denestomed   4.0K Feb 26  2022 Denestomed

drwx------  2 Droutich1970      Droutich1970 4.0K Feb 26  2022 Droutich1970

drwx------  2 Enten1988         Enten1988    4.0K Feb 26  2022 Enten1988

drwx------  2 Estion59          Estion59     4.0K Feb 26  2022 Estion59

drwx------  2 Gulay1979         Gulay1979    4.0K Feb 26  2022 Gulay1979

drwx------  2 Hanceek           Hanceek      4.0K Feb 26  2022 Hanceek

drwx------  2 Haremanse84       Har

### What is located in the unauthentized user's home directory?

In [26]:
sftp = ssh.open_sftp()

dir = "/home/p.dalton/"

stdin, stdout, stderr = ssh.exec_command("ls -la " + dir)

for line in stdout.readlines():
    print(line)

### How long was the login sesssion? 9 Minutes

In [ ]:
stdin, stdout, stderr = ssh.exec_command("last p.dalton")

for line in stdout.readlines():
    print(line)

### Has the user logged in before? Yes

In [ ]:
stdin, stdout, stderr = ssh.exec_command("lastlog | grep p.dalton")

for line in stdout.readlines():
    print(line)

## lsof -p XXXX

In [2]:
lines = []

thePass = getpass(prompt="Please enter your user Password: ")

with open('pid.txt', 'r') as f:
    for line in f:
        lines.append(line)

for item in lines:
    split_lines = item.split(" ")

for t in split_lines:
    # print(t)

    command = "sudo -S lsof -p " + t

    # print(command)

    stdin, stdout, stderr = ssh.exec_command(command)
    stdin.write(thePass + "\n")
    stdin.flush()

    for s in stdout.readlines():
        print(s)

COMMAND    PID USER   FD      TYPE             DEVICE SIZE/OFF    NODE NAME

snapd   519423 root  cwd       DIR                8,1     4096       2 /

snapd   519423 root  rtd       DIR                8,1     4096       2 /

snapd   519423 root  txt       REG                7,0 32457312     151 /snap/snapd/17336/usr/lib/snapd/snapd

snapd   519423 root  mem       REG                8,1   101320   21165 /usr/lib/x86_64-linux-gnu/libresolv-2.31.so

snapd   519423 root  mem       REG                8,1    27080   19483 /usr/lib/x86_64-linux-gnu/libnss_dns-2.31.so

snapd   519423 root  mem       REG                8,1    51832   19616 /usr/lib/x86_64-linux-gnu/libnss_files-2.31.so

snapd   519423 root  mem       REG                8,1  2029560    2575 /usr/lib/x86_64-linux-gnu/libc-2.31.so

snapd   519423 root  mem       REG                8,1   157224   20034 /usr/lib/x86_64-linux-gnu/libpthread-2.31.so

snapd   519423 root  mem       REG                8,1   191472    2570 /usr/lib/x86_6

## Check "bin" directories for malicious files

In [8]:
cmds = ["ls -al -h k /usr/bin/", "ls -al -h k /usr/sbin/", "ls -al -h k /usr/local/bin/", "ls -al -h k /usr/local/sbin/"]

if os.path.exists('bin_results.txt'):
    usrinput = input("Would you like to delete this file? (Y/N) ")
    if usrinput == "Y" or usrinput == "y":
        os.remove("bin_results.txt")

for command in cmds:
    # Get the outpt form the command
    stdin, stdout, stderr = ssh.exec_command(command)

    split_lines = []

    for line in stdout.readlines():
        # print(line.split(' '))
        split_lines.append(line.split(' '))
        
        # Searches and returns results using a regular expression search

    files = []
    split_command = command.split(' ')

    # print(split_lines)
    # loop through split_lines
    for new_line in split_lines:
        # if new_line is longer than 2
        if len(new_line) > 2:
            # if there is a symlink
            if new_line[-2] != '->':
                for item in new_line:
                    # print(item)
                    if '\n' in item:
                        # print(item[0:-2])
                        files.append(item[0:-2])
                        
            else:
                # print(new_line[-3])
                files.append(new_line[-3])

    # for r in files:
    #     print(r)


    for file in files:
        x = re.findall(r'^([a-z]{6,17})$', file)
        for found in x:
            output = split_command[-1] + found

    
    # Get results from stdout
    # lines = stdout.readlines()
    # print(lines)

    # Cover the list to a string
    # output = ''.join(lines)

            # Header output
            sepHeader = '' + '### BEGIN ' + command + ' ###\n\n'
            # Footer
            sepFooter = '' + '### END ' + command + ' ###\n\n'

            # Concatenate the header, output, and footer
            cmd_output = sepHeader + output + sepFooter

            # Sace the cmd_output to a file
            # a is append mode
            with open('bin_results.txt', 'a') as f:
                f.write(output + '\n')

## Run lsof using sudo

In [4]:
thePass = getpass(prompt="Please enter your user Password: ")

stdin, stdout, stderr = ssh.exec_command("sudo -S lsof -i -n | awk '{print $2}'")
stdin.write(thePass + "\n")
stdin.flush()

for line in stdout.readlines():
    print(line)

PID

593

603

603

830

855

855

935

935

561559

600020

600170

600390

600489

600523

600626

600716

600818

600854

600947

600952

601052

601067

601169

601242

601357

601383

601485

601534

601633

601653

601752

601759

601861

601881

601977

601982

602084

602108

602204

602220

602318

602330

602429

602470

602568

602604

602705

602900

603028

603036

603134

603758

603871

604366

604465



## ls p.datlon Folder

In [7]:
stdin, stdout, stderr = ssh.exec_command("ls /home/p.dalton")

for line in stdout.readlines():
    print(line)

aqwerpuqwerj

d.pid

test-lk



## Close connection

In [6]:
ssh.close()

## Reflections

### Based on what you discovered and the information in the threat report, what is your conclusion about the host you analyzed?


### What did you like the most and least about this assignment?

I disliked that the Threat Report Background information really throw me off because of how it was worded and after that it was extremely hard to recenter on the main goal of this assinment. 

### What additional questions do you have?


What is the process name?
egrep

Which user is running the process?
p.datlon

Has the user logged in before via SSH?
yes

When did the user login?
10/25/2022 23:31:31

How long was the login session?
9 Mintues

What is located in the unauthorized user’s home directory?
aqwerpuqwerj
d.pid
test-lk

Download any suspicious files.
